In [1]:
from pathlib import Path

from preprocessing import drop_cols, date_split

In [2]:
DATA_DIR = Path("../data")

In [3]:
drop_cols(file=DATA_DIR / "raw/data.csv",
          file_out=DATA_DIR / "interim/data.csv",
          cols=["banner_id0", "banner_id1", "rate0", "rate1", "g0", "g1",
                "coeff_sum0", "coeff_sum1"])

!head {DATA_DIR / "interim/data.csv"}

date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1
2021-09-27 00:06:50.000000,5,5,2377014068362699676,0,2,2,1,1
2021-09-27 00:07:34.000000,6,6,6863358899511896876,0,3,0,1,1
2021-09-27 00:08:49.000000,7,7,2876502170484631685,0,4,1,1,1
2021-09-27 00:09:08.000000,8,8,5839858970958967275,0,4,3,1,1


In [4]:
date_split(file=DATA_DIR / "interim/data.csv",
           output_dir=DATA_DIR / "interim",
           test_date="2021-10-02")

!head {DATA_DIR / "interim/train.csv"}

date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1
2021-09-27 00:06:50.000000,5,5,2377014068362699676,0,2,2,1,1
2021-09-27 00:07:34.000000,6,6,6863358899511896876,0,3,0,1,1
2021-09-27 00:08:49.000000,7,7,2876502170484631685,0,4,1,1,1
2021-09-27 00:09:08.000000,8,8,5839858970958967275,0,4,3,1,1
